In [2]:
import pandas as pd
import nba_api

In [44]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from nba_api.stats.endpoints import playercareerstats, leaguegamelog
from nba_api.stats.static import players
import plotly.graph_objects as go

def get_player_id(player_name):
    # 選手名からプレイヤーIDを取得
    player = players.find_players_by_full_name(player_name)
    if player:
        return player[0]['id']
    else:
        raise ValueError("Player not found")

def get_player_stats(player_id):
    # プレイヤーのキャリアスタッツを取得
    career = playercareerstats.PlayerCareerStats(player_id=player_id).get_data_frames()[0]
    career = career[['SEASON_ID', 'PTS']]
    # 同じSEASON_IDが複数ある場合、PTSを合計する
    career = career.groupby('SEASON_ID', as_index=False).sum()

    # SEASON_IDを数値型に変換
    career['SEASON_ID'] = career['SEASON_ID'].str[:4].astype(int)
    career = career.sort_values('SEASON_ID').reset_index(drop=True)

    # シーズンのギャップを埋める
    full_seasons = pd.DataFrame({'SEASON_ID': range(career['SEASON_ID'].min(), career['SEASON_ID'].max() + 1)})
    career = pd.merge(full_seasons, career, on='SEASON_ID', how='left').fillna(0)
    return career

def plot_cumulative_points_comparison(player_name1, player_name2):
    player_id1 = get_player_id(player_name1)
    player_stats1 = get_player_stats(player_id1)

    player_id2 = get_player_id(player_name2)
    player_stats2 = get_player_stats(player_id2)

    # 累積得点を計算
    player_stats1['CUMULATIVE_PTS'] = player_stats1['PTS'].cumsum()
    player_stats2['CUMULATIVE_PTS'] = player_stats2['PTS'].cumsum()

    # シーズンをキャリア年数に変換
    max_career_length = max(len(player_stats1), len(player_stats2))
    player_stats1['CAREER_YEAR'] = range(1, len(player_stats1) + 1)
    player_stats2['CAREER_YEAR'] = range(1, len(player_stats2) + 1)

    # 補完処理（短い方をゼロ埋めする）
    if len(player_stats1) < max_career_length:
        filler = max_career_length - len(player_stats1)
        additional_rows = {
            'CAREER_YEAR': range(len(player_stats1) + 1, max_career_length + 1),
            'CUMULATIVE_PTS': 0 * filler
        }
        player_stats1 = pd.concat([player_stats1, pd.DataFrame(additional_rows)], ignore_index=True)

    if len(player_stats2) < max_career_length:
        filler = max_career_length - len(player_stats2)
        additional_rows = {
            'CAREER_YEAR': range(len(player_stats2) + 1, max_career_length + 1),
            'CUMULATIVE_PTS': 0 * filler
        }
        player_stats2 = pd.concat([player_stats2, pd.DataFrame(additional_rows)], ignore_index=True)

    # プロット
    fig = go.Figure()

    fig.add_trace(go.Bar(
        x=player_stats1['CAREER_YEAR'],
        y=player_stats1['CUMULATIVE_PTS'],
        name=f'{player_name1} (Cumulative Points)',
        marker=dict(color='gold', line=dict(color='black', width=1))
    ))

    fig.add_trace(go.Bar(
        x=player_stats2['CAREER_YEAR'],
        y=player_stats2['CUMULATIVE_PTS'],
        name=f'{player_name2} (Cumulative Points)',
        marker=dict(color='purple', line=dict(color='black', width=1))
    ))

    fig.update_layout(
        title=dict(
            text=f'Cumulative Points Comparison: {player_name1} vs {player_name2}',
            font=dict(size=20, color='black'),
            x=0.5
        ),
        xaxis=dict(
            title='Career Year',
            titlefont=dict(size=16, color='black'),
            tickfont=dict(size=14, color='black')
        ),
        yaxis=dict(
            title='Cumulative Points',
            titlefont=dict(size=16, color='black'),
            tickfont=dict(size=14, color='black')
        ),
        legend=dict(
            font=dict(size=14),
            bgcolor='rgba(255,255,255,0.8)',
            bordercolor='black',
            borderwidth=1
        ),
        barmode='group',
        plot_bgcolor='white',
        grid=dict(
            rows=1, columns=1,
            pattern='independent'
        )
    )

    fig.show()

# 使用例
plot_cumulative_points_comparison("James Harden", "Russell Westbrook")




ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

In [39]:
a = get_player_id("Kevin Durant")
b = get_player_stats(a)
print(b)

    SEASON_ID     PTS
0        2007  1624.0
1        2008  1871.0
2        2009  2472.0
3        2010  2161.0
4        2011  1850.0
5        2012  2280.0
6        2013  2593.0
7        2014   686.0
8        2015  2029.0
9        2016  1555.0
10       2017  1792.0
11       2018  2027.0
12       2019     0.0
13       2020   943.0
14       2021  1643.0
15       2022  2732.0
16       2023  2032.0
17       2024   422.0
